In [1]:
## Import the required libraries and read the data into memory
## Rename the column names from spaces to underscores

import duckdb
import pandas as pd
from plotnine import (
    ggplot,
    aes,
    geom_bar,
    theme,
    element_text
)

df = pd.read_csv('./data/Trade_DetailedTradeMatrix.csv', encoding='latin-1')

colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)

df.columns = colnames

In [56]:
## This function calculates the trade dependency of a country by year by total import value.
## It fetches the top 20 countries by import value that the selected country imports from.
## For example, if you use trade_dependency('United Arab Emirates', 2021)
## You will get the top 20 countries by import value (USD $) that the U.A.E imports from
## In the year 2021. 

def trade_dependency(country, year):
    
    dataframe = duckdb.sql('''
    select Partner_Countries,
    sum(Value) as import_value,
    import_value /
        (select
            sum(Value)
            from df
            where Year = {year}
            and Element = 'Import Value'
            and Reporter_Countries = '{country}'
        ) as imp_prop
    from df
    where Year = {year}
    and Element = 'Import Value'
    and Reporter_Countries = '{country}'
    group by Partner_Countries
    having imp_prop > 0
    order by imp_prop desc;
    '''.format(year = year, country = country)).fetchdf().head(21)

    return dataframe

trade_dependency('Oman', 2018)

,Partner_Countries,import_value,imp_prop
0,United Arab Emirates,1824275.0,0.453535
1,India,397051.0,0.098711
2,Saudi Arabia,240000.0,0.059667
3,Pakistan,183415.0,0.045599
4,Somalia,141632.0,0.035211
5,Russian Federation,97926.0,0.024345
6,Indonesia,86039.0,0.021390
7,Iran (Islamic Republic of),76016.0,0.018898
8,Brazil,68755.0,0.017093
9,Argentina,62299.0,0.015488


In [54]:
## This function finds the trade dependants by value (USD $ of a country).
## For example, using trade_dependants('United Arab Emirates', 2021)
## Finds the top 20 countries that depend on the U.A.E
## The dependency for country A on country B is quantified by taking
## The proportion of imports from country B to country A over the total
## Imports to country A:
##
## dep_ratio = (Import value from B to A) / (Total import value to country A)

def trade_dependants(country, year):

    query = '''
    drop table if exists top_imports;
    drop table if exists total_imports;

    create table top_imports as
    select Reporter_Countries,
        sum(Value) as imports
        from df
        where Year = {year}
        and Element = 'Import Value'
        and Partner_Countries = '{country}'
        group by Reporter_Countries
        having imports > 0
        order by imports desc;

    create table total_imports as
    select Reporter_Countries,
        sum(Value) as import_value
        from df
        where Year = {year}
        and Element = 'Import Value'
        group by Reporter_Countries
        having import_value > 0
        order by import_value desc;
    
        drop table if exists imps;
        drop table if exists dependants;

        create table imps as
            select * from top_imports
            left join total_imports
            on top_imports.Reporter_Countries = total_imports.Reporter_Countries
            order by import_value desc;

            select Reporter_Countries, imports,
                imports / import_value as dep_ratio
                from imps
                order by dep_ratio desc;
        '''.format(year = year, country = country)

    dependants = duckdb.sql(query).fetchdf()
    
    return dependants

trade_dependants('United Arab Emirates', 2021)

,Reporter_Countries,imports,dep_ratio
0,Oman,1864287.0,3.505727e-01
1,Maldives,118694.0,2.034700e-01
2,Afghanistan,420899.0,1.863838e-01
3,Seychelles,32486.0,1.848579e-01
4,Bahrain,307722.0,1.759797e-01
...,...,...,...
153,Costa Rica,12.0,4.863604e-06
154,Honduras,11.0,4.849190e-06
155,Finland,13.0,2.104723e-06
156,Estonia,1.0,5.591006e-07
